In [31]:
import pandas as pd
import xlsxwriter

nutri_chart = pd.read_excel("/home/hemant/Documents/Mani_nutri/nutri_chart.xlsx")
nutri_templ = pd.read_excel("/home/hemant/Documents/Mani_nutri/nutri_template.xlsx")
from_site = pd.read_csv("/home/hemant/Documents/Mani_nutri/from_site.csv")
nutri_chart = nutri_chart.fillna("-")

days = {1: "पहिला", 2: "दुसरा", 3: "तिसरा" }
menu = {1:"चहा",2:"ब्रेड",3:"बिस्कीट",4:"दूध",5:"उपमा",6:"पोहे",7:"शिरा",8:"दहीवडे",9:"उसाचा रस",10:"वाटाणा उसळ",11:"मटकीची उसळ",12:"चण्याची उसळ",
        13:"साबुदाणा उसळ",14:"कढी",15:"पापड",16:"भाकर",17:"पोळी",18:"भात",19:"वरण",20:"खिचडी",21:"बेसन",22:"पत्ता कोबी",23:"मेथीची भाजी",24:"डाळ पालक",
        25:"बटाटा वडी",26:"पालक वरण",27:"आलूची भाजी",28:"टमाटर चटणी",29:"वांग्याचं भरीत",30:"वांग्याची भाजी",31:"अंड्याची भाजी"}

# Create a workbook and add a worksheet.
file_name = '/home/hemant/Documents/Mani_nutri/Manisha_Kshirsagar_nutrition_chart.xlsx'
writer = pd.ExcelWriter(file_name,engine='xlsxwriter')   
row = 0

for site in from_site.itertuples():
    test_1 = list(str(site.Menu_1).strip().split(","))
    test_2 = list(str(site.Menu_2).strip().split(","))
    
    input_menu = []
    for t in test_1:
        for key, val in menu.items():
            if t == val:
                input_menu.append(key)
    for t in test_2:
        for key, val in menu.items():
            if t == val:
                input_menu.append(key)
    menu_col=[]
    menu_df = pd.merge(nutri_templ, nutri_chart[["पोषक घटक"]+[menu[x] for x in input_menu]], on="पोषक घटक")
    menu_df['Total']=round(menu_df.iloc[:,2:].sum(axis=1),2)
    menu_col.append('Document_No')
    menu_col.append('Day_No')
    menu_col.append('Total')
    for col in menu_df.columns.tolist():
        if col != 'Total':
            menu_col.append(col)    
    menu_df = menu_df.assign(Document_No=int(site.Document_No))
    menu_df = menu_df.assign(Day_No=int(site.Day_No))
    
    df = pd.DataFrame([])
    df["Document_No"] = menu_df["Document_No"]
    df["Day_No"] = menu_df["Day_No"]
    df["Total"] = menu_df["Total"]
    df["पोषक घटक"] = menu_df["पोषक घटक"]
    
    df = pd.merge(df,menu_df.iloc[:,0:-3], on="पोषक घटक")
    del df["अ. क्र."]
    df.to_excel(writer,sheet_name='Manisha_Kshirsagar',startrow=row , startcol=0)   
    row = row + len(df.index) + 1 + 1     
writer.save()
